<a href="https://colab.research.google.com/github/syedzaidi-kiwi/RAG-frameworks-Kiwitech/blob/main/Mistral_qdrant_getting_ready_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qdrant-client mistralai -qq

In [ ]:
from mistralai.client import MistralClient
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance
collection_name = "example_collection"

In [ ]:
MISTRAL_API_KEY = "your_mistral_api_key"

In [ ]:
search_client = QdrantClient(":memory:")
mistral_client = MistralClient(api_key=MISTRAL_API_KEY)
texts = [
    "Qdrant is the best vector search engine!",
    "Loved by Enterprises and everyone building for low latency, high performance, and scale.",
]

In [ ]:
result = mistral_client.embeddings(
    model="mistral-embed",
    input=texts,
)

In [ ]:
len(result.data[0].embedding)

1024

In [ ]:
points = [
    PointStruct(
        id=idx,
        vector=response.embedding,
        payload={"text": text},
    )
    for idx, (response, text) in enumerate(zip(result.data, texts))
]

In [ ]:
search_client.create_collection(collection_name, vectors_config=
    VectorParams(
        size=1024,
        distance=Distance.COSINE,
    )
)
search_client.upsert(collection_name, points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
search_client.search(
    collection_name=collection_name,
    query_vector=mistral_client.embeddings(
        model="mistral-embed", input=["What is the best to use for vector search scaling?"]
    ).data[0].embedding,
)

[ScoredPoint(id=0, version=0, score=0.7706972129206147, payload={'text': 'Qdrant is the best vector search engine!'}, vector=None, shard_key=None),
 ScoredPoint(id=1, version=0, score=0.6887654944454833, payload={'text': 'Loved by Enterprises and everyone building for low latency, high performance, and scale.'}, vector=None, shard_key=None)]